In [ ]:
# keep only subsiding polygon pixels from cum_nozeros_no_nans
# convert polygons shape into a geopandas dataframe
frames_gdf["subsiding_restored_signals"] = ""
frames_gdf["restored_signals_3d"] = ""

gdf_polygons = gpd.read_file(subs_poly_path)

for index, row in frames_gdf.iterrows():
    frame = row['frame']
    non_nan_ind = row['non_nan_ind']
    non_zero_ind = row['non_zero_ind']
    restored_signals_outer = row['restored_signals']
    cum = row['cum']
    lon_plot = row['lon']
    lat_plot = row['lat']

    common_indices = np.intersect1d(non_nan_ind, non_zero_ind)

    cum_with_nans = np.full((cum.shape[0], cum.shape[1], cum.shape[2]), np.nan)

    # Create meshgrid of lon and lat
    lon, lat = np.meshgrid(lon_plot, lat_plot)

    # Flatten lon and lat
    lon_1d = lon.flatten()
    lat_1d = lat.flatten()

    # Create a GeoDataFrame for the flattened lon and lat
    geometry = [Point(lon, lat) for lon, lat in zip(lon_1d, lat_1d)]
    gdf_points = gpd.GeoDataFrame(geometry=geometry, columns=['geometry'])
    gdf_points.crs = "EPSG:4326"
    gdf_points['Latitude'] = lat_1d
    gdf_points['Longitude'] = lon_1d

    # Perform Spatial Join
    joined = gpd.sjoin(gdf_points, gdf_polygons, predicate='within')

    extracted_coordinates = joined[['Latitude', 'Longitude']]

    extracted_indices = joined.index

    # Create a boolean mask for lon and lat arrays
    mask = np.isin(np.arange(lon.size), extracted_indices)

    list = []
    three_d_list = []
    counter = 0

    ## this bit of code takes restored signals from ICA and populates the restored data onto non nan/nonzero indices within cum
    ## this needs to be done post ICA still
    ## this restored 3d array is then masked using the polygon outlines to keep only restored signals in the subsiding regions
    ## this is done for 2d and 3d cum arrays
    for restored_signal in restored_signals_outer:
        # Convert common_indices to 3D indices i.e. flat indices into a tuple for 3d array
        indices_3d = np.unravel_index(common_indices, cum.shape[1:])

        # Create a copy of cum_with_nans to work with for each restored_signal
        cum_with_nans_copy = cum_with_nans.copy()
        
        # need to make sure a different time series assigned to timestep
        for j in range(restored_signal.shape[0]):
            # Assign values from the restored signal to non-NaN positions
            cum_with_nans_copy[j, indices_3d[0], indices_3d[1]] = restored_signal[j,:]

        # reshape cum_with_nans into time x pixels
        cum_with_nans_pix = cum_with_nans_copy.reshape(cum.shape[0], cum.shape[1] * cum.shape[2])

        #reshape cum_with_nans_pix into 3d to save signals
        restored_signal_3d = cum_with_nans_pix.reshape(cum.shape[0], cum.shape[1], cum.shape[2])
        three_d_list.append(restored_signal_3d)

        # mask cum_with_nans_pix with extracted indices
        masked_cum_with_nans_pix = cum_with_nans_pix * np.where(mask == 0, np.nan, 1)
        list.append(masked_cum_with_nans_pix)
        
    frames_gdf.at[index, 'subsiding_restored_signals'] = list
    frames_gdf.at[index, "restored_signals_3d"] = three_d_list

In [ ]:
# remove nans and zeros from each subsiding restored signal
frames_gdf["subsiding_no_nans"] = ""

for index, row in frames_gdf.iterrows():
    frame = row['frame']
    subsiding_restored = row['subsiding_restored_signals']
    no_nans_store = []
    for i, restored_signal in enumerate(subsiding_restored, start=0):
        # Print how many NaNs there are
        nan_indices = np.argwhere(np.isnan(restored_signal))

        # find columns containing NaNs and zeroes
        nan_pixels = np.any(np.isnan(restored_signal), axis=0)

        # create a new data array without nan columns (pixels)
        subsiding_no_nans = restored_signal[:, ~nan_pixels]
        no_nans_store.append(subsiding_no_nans)
    frames_gdf.at[index, 'subsiding_no_nans'] = no_nans_store